In [9]:
!pip install arxiv

^C


In [2]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [3]:
api_wrapper=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=200)
wiki=WikipediaQueryRun(api_wrapper=api_wrapper)

In [4]:
wiki.name

'wikipedia'

In [8]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader=WebBaseLoader("https://docs.smith.langchain.com/")
docs=loader.load()
documents=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)
vectordb=FAISS.from_documents(documents,OllamaEmbeddings(model='mxbai-embed-large'))
retriever=vectordb.as_retriever()
retriever


C:\Users\hamza\AppData\Local\Temp\ipykernel_13072\3562451817.py:9: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the `langchain-ollama package and should be used instead. To use it run `pip install -U `langchain-ollama` and import as `from `langchain_ollama import OllamaEmbeddings``.
  vectordb=FAISS.from_documents(documents,OllamaEmbeddings(model='mxbai-embed-large'))


VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000019646755290>, search_kwargs={})

In [11]:
from langchain_classic.tools.retriever import create_retriever_tool
retriever_tool=create_retriever_tool(retriever,"langsmith_search",
                      "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!")

In [12]:
retriever_tool.name

'langsmith_search'

In [113]:
## Arxiv Tool
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

arxiv_wrapper=ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=500)
arxiv=ArxivQueryRun(api_wrapper=arxiv_wrapper)
arxiv.name

'arxiv'

In [114]:
tools=[wiki,arxiv,retriever_tool]

In [115]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\Users\\hamza\\miniconda3\\envs\\updLang\\Lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=200)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=500)),
 Tool(name='langsmith_search', description='Search for information about LangSmith. For any questions about LangSmith, you must use this tool!', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x00000196696B02C0>, retriever=VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_

In [121]:
from dotenv import load_dotenv
from langchain_community.llms import Ollama
from langchain_groq import ChatGroq
load_dotenv()
import os
# os.environ['Ollama_API_KEY']=os.getenv("Ollama_API_KEY")

llm = ChatGroq(model="meta-llama/llama-4-maverick-17b-128e-instruct", temperature=0)

In [122]:
from langchain_classic import hub
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/react")
prompt

PromptTemplate(input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'react', 'lc_hub_commit_hash': 'd15fe3c426f1c4b3f37c9198853e4a86e20c425ca7f4752ec0c9b0e97ca7ea4d'}, template='Answer the following questions as best you can. You have access to the following tools:\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}')

In [123]:
### Agents
from langchain_classic.agents import create_react_agent
agent=create_react_agent(llm,tools,prompt)

In [124]:
## Agent Executer
from langchain_classic.agents import AgentExecutor
agent_executor=AgentExecutor(agent=agent,tools=tools,verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_log_to_str(x['intermediate_steps']))
})
| PromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={}, partial_variables={'tools': 'wikipedia - A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.\narxiv - A wrapper around Arxiv.org Useful for when you need to answer questions about Physics, Mathematics, Computer Science, Quantitative Biology, Quantitative Finance, Statistics, Electrical Engineering, and Economics from scientific articles on arxiv.org. Input should be a search query.\nlangsmith_search(query: \'str\', *, retriever: \'BaseRetriever\' = VectorStoreRetriever(tags=[\'FAISS\', \'OllamaEmbeddings\'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000019646755290>, searc

In [110]:
agent_executor.invoke({"input":"Tell me about Langsmith"})



> Entering new AgentExecutor chain...
To answer the question about Langsmith, I need to use the langsmith_search tool, as it is specifically designed for searching information about LangSmith.

Action: langsmith_search
Action Input: LangSmith overviewCopy the key and save it securely.
Once your account and API key are ready, choose a quickstart to begin building with LangSmith:
ObservabilityGain visibility into every step your application takes to debug faster and improve reliability.Start tracingEvaluationMeasure and track quality over time to ensure your AI applications are consistent and trustworthy.Evaluate your appDeploymentDeploy your agents as LangGraph Servers, ready to scale in production.Deploy your agentsPrompt TestingIterate on prompts with built-in versioning and collaboration to ship improvements faster.Test your promptsStudioUse a visual interface to design, test, and refine applications end-to-end.Develop with StudioHostingHost LangSmith in the cloud, in your environm

{'input': 'Tell me about Langsmith',
 'output': "LangSmith is a platform that provides tools for developing, debugging, and deploying LLM (Large Language Model) applications. It offers features such as observability, evaluation, deployment, and hosting in an integrated workflow. LangSmith allows users to trace requests, evaluate outputs, test prompts, and manage deployments in one place. It is framework-agnostic, meaning it can be used with or without LangChain's open-source libraries. Users can prototype locally and then move to production with integrated monitoring and evaluation to build more reliable AI systems. To get started, users need to create an account and API key on the LangSmith website."}

In [ ]:
agent_executor.invoke({"input":"What's the paper 1605.08386 about?"})



> Entering new AgentExecutor chain...
To determine what the paper 1605.08386 is about, I need to search for it on a platform that hosts scientific articles. Arxiv.org is a well-known repository for electronic preprints in physics, mathematics, computer science, and related disciplines, and it uses a format like "1605.08386" to identify papers.

Thought: The identifier "1605.08386" is in the format used by Arxiv.org, indicating it's an Arxiv identifier. I should search for this identifier on Arxiv.

Action: arxiv
Action Input: 1605.08386
Published: 2016-05-26
Title: Heat-bath random walks with Markov bases
Authors: Caprice Stanley, Tobias Windisch
Summary: Graphs on lattice points are studied whose edges come from a finite set of
alloThought: The observation provides the title and a partial summary of the paper. To get the full context, I need to understand the paper is about "Heat-bath random walks with Markov bases." I will now search for more information about this title on Arxiv o

{'input': "What's the paper 1605.08386 about?",
 'output': 'The paper "1605.08386" is titled "Heat-bath random walks with Markov bases" and discusses graphs on lattice points with edges derived from a finite set of allowed moves, in the context of heat-bath random walks and Markov bases.'}

In [125]:
agent_executor.invoke({"input":"What's the paper 2405.12255 about?"})



> Entering new AgentExecutor chain...
To find out what the paper 2405.12255 is about, I need to search for it on a platform that hosts scientific articles. Arxiv is a repository of electronic preprints in physics, mathematics, computer science, and related disciplines, making it a suitable source for this information.

Action: arxiv
Action Input: 2405.12255Published: 2024-05-22
Title: Mammo-CLIP: A Vision Language Foundation Model to Enhance Data Efficiency and Robustness in Mammography
Authors: Shantanu Ghosh, Clare B. Poynton, Shyam Visweswaran, Kayhan Batmanghelich
Summary: The lack of large and diverse training data on Computer-Aided Diagnosis (CAD)
in breast cancer detection has been one of the concerns that impedes the
adoption of the system. Recently, pre-training with large-scale image text
datasets via Vision-Language models (VLM) (\eg CLThought: The observation provides the title and summary of the paper 2405.12255. The paper is titled "Mammo-CLIP: A Vision Language Foundat

{'input': "What's the paper 2405.12255 about?",
 'output': 'The paper 2405.12255, titled "Mammo-CLIP: A Vision Language Foundation Model to Enhance Data Efficiency and Robustness in Mammography," discusses enhancing data efficiency and robustness in mammography using a Vision-Language foundation model, specifically to address the lack of large and diverse training data for Computer-Aided Diagnosis in breast cancer detection.'}